<a href="https://colab.research.google.com/github/AviFeygin/EECS-4080/blob/main/Israpol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import torch
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import keras as ker
import seaborn as sea
import xlrd as xlr
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import theano
import statsmodels.api as stm

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

#ignore all warnings
import warnings
warnings.filterwarnings('ignore')
sea.set()

import itertools
!pip install --upgrade xlrd

from google.colab import data_table
data_table.enable_dataframe_formatter()
from vega_datasets import data

verbose = 0

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading Text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = '/content/drive/MyDrive/Polls.xls'
polls = pd.read_excel(data)
polls_2021 = pd.read_excel(data, sheet_name=0, index_col=0)

In [ ]:
print("polls_2021 shape:")
print(polls_2021.shape)
just_polls = polls_2021.iloc[1:161, 4:25]
print("just polls")
#fixing the float errors
just_polls['Days Before election'] = just_polls['Days Before election'].round().astype('int')
display(just_polls)
number_of_parties = 14
number_of_blocs = 4

polls_2021 shape:
(159, 23)
just polls


,Days Before election,Likud,Yesh Atid,Blue & White,Joint List,Shas,UTJ,Yisrael Beiteinu,Meretz,Ra'am,Yamina,New Hope,Labor,Religious Zionist,NEP,Right Wing,Left Wing,religious,Arab
Date,,,,,,,,,,,,,,,,,,,
2021-03-20 00:00:02,3,31,19,4.0,8,8,7,7,4.0,4.0,9,9,5.00,5.00,1.68,61.00,28.00,15,16.0
2021-03-19 00:00:03,4,30,18,4.0,8,8,7,6,4.0,4.0,10,10,6.00,5.00,0.00,61.00,28.00,15,16.0
2021-03-19 00:00:04,4,32,18,4.0,8,8,7,7,4.0,4.0,9,9,6.00,4.00,1.56,61.00,28.00,15,16.0
2021-03-19 00:00:05,4,31,19,4.0,8,8,7,7,4.0,4.0,9,9,5.00,5.00,1.68,61.00,28.00,15,16.0
2021-03-18 00:00:06,5,30,19,5.0,8,8,6,8,4.0,4.0,10,8,5.00,5.00,0.00,61.00,29.00,14,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-17 00:02:35,279,41,15,11.0,16,10,7,8,5.0,0.0,7,0,1.32,0.60,0.00,56.60,27.32,17,21.0
2020-06-17 00:02:36,279,41,15,10.0,16,9,7,8,6.0,0.0,8,0,2.76,1.68,0.00,58.68,27.76,16,22.0
2020-06-11 00:02:37,285,41,16,13.0,15,9,8,7,5.0,0.0,6,0,0.00,0.00,0.00,54.00,29.00,17,20.0


Parties

In [ ]:
likud = just_polls[['Days Before election','Likud']]
#display(likud)
yesh_atid = just_polls[['Days Before election','Yesh Atid']]
blue_and_white = just_polls[['Days Before election','Blue & White']]
joint_list = just_polls[['Days Before election','Joint List']]
shas = just_polls[['Days Before election','Shas']]
utj = just_polls[['Days Before election','UTJ']]
beiteinu = just_polls[['Days Before election','Yisrael Beiteinu']]
meretz = just_polls[['Days Before election','Meretz']]
raam = just_polls[['Days Before election','Ra\'am']]
yamina = just_polls[['Days Before election','Yamina']]
new_hope = just_polls[['Days Before election','New Hope']]
Labor = just_polls[['Days Before election','Labor']]
religious_zionist = just_polls[['Days Before election','Religious Zionist']]


Blocs


In [ ]:
right_wing = just_polls[['Days Before election','Right Wing']]
left_wing = just_polls[['Days Before election','Left Wing']]
religious = just_polls[['Days Before election','religious']]
arab = just_polls[['Days Before election','Arab']]

Defining a LOESS function
it will take in a panda frame column and return the loess column

In [ ]:
#Frame must be a column dataframe with time indexing, delta is an integer that should be the time interval and days which is unique days of the set
#returns new frame with loess and same time indexing
# freame.loc[frame['set_of_numbers'] <= 4, 'equal_or_lower_than_4?'] = 'True' 
# frame.loc[frame['set_of_numbers'] > 4, 'equal_or_lower_than_4?'] = 'False' 
verbose =0
just_polls['Days Before election'].nunique()
def LOESS(frame, delta = 2, days = 1, name = 'Likud'):
  display(frame.shape)
  delta = (delta-1)/2
  length = len(frame.index)
  Loess_frame = pd.DataFrame()
  temp = pd.DataFrame()
  #above_35 = titanic[titanic["Age"] > 35]
  for i in range(3, frame['Days Before election'].max()):
   temp = frame[(frame['Days Before election'] <= (i+delta)) & (frame['Days Before election'] >= (i-delta)) ]
   if len(temp.index)>0:
    x = temp['Days Before election'].values.reshape(-1,1)
 
    y = temp[name].values.reshape(len(temp.index),1)

    if (verbose == 1):
       print("i: ")
       display(i)
       print("y: ")
       display(y)
       print("x: ")
       display(x)


    model = linear_model.LinearRegression().fit(x, y)
    linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
    linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
    display(model.predict([[i]]))
    Loess_frame.append([i, model.predict([[i]])])
    display(Loess_frame)
  return Loess_frame

likud_Loess = LOESS(likud, 3, just_polls['Days Before election'].nunique(), 'Likud')
display(likud_Loess)



(158, 2)

i: 


3

y: 


array([[31],
       [30],
       [32],
       [31]])

...

x: 


array([[301]])

array([[41.]])

""


""


Defining the LOWESS function

In [ ]:
delta=3
length = just_polls['Days Before election'].max()
indeces = just_polls['Days Before election']
delta = delta/2
loess_data = zeroes
  #frame is the pandas dataframe, range is the delta of spread. higher delta means more days. length is the total number of days before an election since this assumes some polls may overlap.

for i in range(length):
  temp_frame  = just_polls;
  for column in just_polls[2:]:
    temp_frame[column, just_polls['Days Before Election']>i - delta and just_polls['Days Before Election'] < i+delta ]



NameError: ignored

In [ ]:
#[:,-1] rows and columns

just_polls_parties = just_polls.iloc[:, 0:13]
just_polls_blocks = just_polls.iloc[:,14:18]
just_polls_blocks.plot(figsize=(20, 30, ), style =".")

#You pass the wanted axis to the ax argument
df.plot(kind='scatter', x='x', y='y',title="Nice title", grid=True,fontsize=10, ax=ax) 
ax.plot(line, x_line, zorder=-1)

In [ ]:
#Loess and lowess

for i in range(length(just_polls))
  display(i)

In [ ]:
covariance = just_polls.corr()
print("covariance")
display(covariance)
heat_map = sea.heatmap(df.corr(), covariance,annot=True, cmap = "RdBu")
plt.figure(figsize = (30,15))
sea.heatmap()

In [ ]:
correlation_3_day = pd.read_excel(data, sheet_name=1, index_col=0)
display(correlation_3_day)

3 day rolling average

In [ ]:
correlation_3_day = correlation_3_day.iloc[1:159, 25:35]
display(correlation_3_day)


In [ ]:
correlation_3_poll = pd.read_excel(data, sheet_name=2, index_col=0)
correlation_3_poll = correlation_3_poll.iloc[444:454, 1:19]

heat_map2 = sea.heatmap(correlation_3_day, annot=True, cmap = "RdBu");
plt.show()
heat_map3 = sea.heatmap(correlation_3_poll, annot=True, cmap = "RdBu");
plt.show()
# temp = just_polls[1:1, 1:1]
# start, stop =1;
stm.nonparametric.lowess()


